<a href="https://colab.research.google.com/github/liuying1201/Credit-Risk-Prediction-German-Bank/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning german bank credit risk prediction
In this project, I will build a <b>machine learning model to predict credit risk of German Bank's customers</b>. It involves <b>supervised learning (using a labeled training set) for classification</b>, where the <b>target</b> is <b>1</b> if the <b>customer</b> represents a <b>bad risk</b>, and <b>0</b> if he represents a <b>good risk</b>.

I will use the following <b>pipeline:</b>

<b>1. Define the business problem.</b><br>
<b>2. Collect the data and get a general overview of it.</b><br>
<b>3. Split the data into train and test sets.</b><br>
<b>4. Explore the data (exploratory data analysis)</b><br>
<b>5. Data cleaning and preprocessing.</b><br>
<b>6. Model training, comparison, selection and tuning.</b><br>
<b>7. Final production model testing and evaluation.</b><br>
<b>8. Conclude and interpret the model results.</b><br>
<b>9. Deploy.</b><br>

In <b>this notebook</b>, I will perform <b>machine learning modelling, covering steps 5 to 8 of the pipeline above</b>. The main <b>objective</b> here is to <b>build a model that can predict as many as possible bad risk customers, thus recall is the metric of interest</b>. Once I build this model, the <B>bank can make informed decisions that balance profit generation with prudent risk management</b>, ultimately benefiting both the company and its customers. Furthermore, I will approach these steps in more detail below, with an overview of them and the explanation of why I am making each decision.

### Importing the libraries

In [1]:
# Data manipulation and visualization.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.base import clone

# Fixing bayesian search error.
np.int = int

# Filter warnings.
import warnings
warnings.filterwarnings('ignore')

# Utils
from notebooks.modelling_utils import *

ModuleNotFoundError: No module named 'category_encoders'

### Reading the dataset

In [ ]:
df = pd.read_csv('data/german_credit_data.csv')
# Dropping irrelevant column.
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.head()

Sex and Risk can be expressed as binary features. Thus, I will set good as 1 and bad as 0 in Risk. In sex, I will set male as 1 and female as 0.

In [ ]:
df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
df['Risk'] = df['Risk'].map({'bad': 1, 'good': 0})

### Split the data into train and test sets
- First of all, I will split the data into train and test sets.
- Test set is supposed to be data the model has never seen before.
- This split will avoid data leakage, which occurs when information from the test set or future data inadvertently leaks into the training process, leading to over-optimistic performance estimate, compromising the model's ability to generalize to new, unseen data.
- A good practice to protect the model against data leakage is perform fit_transform on training data and just transform on test data when applying preprocessing steps after the split.
- I will specify stratify=y so that the train_test_split function ensures that the splitting process maintains the same percentage of each target class in both the training and testing sets. This is particularly useful when dealing with imbalanced datasets, which is the case, as there are more good risk customers.

In [ ]:
X = df.drop(columns=['Risk'])
y = df['Risk'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
print(f'Train predictor dataset shape: {X_train.shape}.')
print(f'Train target dataset shape: {y_train.shape}.')
print(f'Test predictor dataset shape: {X_test.shape}.')
print(f'Test target dataset shape: {y_test.shape}.')

In [ ]:
print(f'Train target classes proportions: ')
print(y_train.value_counts(normalize=True))
print(f'\nTest target classes proportions:')
print(y_test.value_counts(normalize=True))

The train-test-split was succesfull and each target class proportion was preserved in both sets.

# 5. Data cleaning and preprocessing
In order to fit machine learning algorithms, it is necessary to apply some transformations to the data.

- <b>Impute missing values:</b> There are some missing values in Saving accounts and Checking account. I will replace them with mode. Dropping the null values is not a good choice because I have a little amount of data (just 1000 rows).<br>
- <b>Outliers:</b> In the eda step, I investigated the outliers and saw that they don't represent inconsistent information like measurement errors. Thus, considering this and the fact that I have a little amount of data, I will not drop or treat any.<br>
- In order to train tree-based algorithms, it is not necessary to scale the data. However, as I intend to test a different algorithms, I will apply standard scaler on numerical and categorical features (after encoding). By doing this, it will be possible to compare all models at once.
- <b>Numerical features:</b> Some algorithms are sensitive to feature scaling because they use distance calculations or optimizations like gradient descent for being trained, which are affected by scale. Thus, I will apply StandardScaler on numeric attributes.<br>
- <b>Categorical features:</b> Machine learning algorithms make mathematical calculations, so it's necessary to convert the categoric attributes to numeric by applying encoding techniques. Saving accounts and Checking account present ordinal relationships. Thus, I chose OrdinalEncoder for encoding them. Moreover, I chose TargetEncoder for the other categorical features because OneHotEncoder would increase dimensionality (creating dummy variables). As I have a little amount of data, increase dimensionality would favor overfitting, compromising my model's generalization ability. After all these variables are encoded/converted to numeric, I will apply StandardScaler because some algorithms are sensitive to scale. StandardScaler is more robust to outliers than MinMaxScaler, that's why I chose it.<br>

In [ ]:
X_train.head()

Checking missing values.

In [ ]:
X_train.isna().sum()

Checking numerical features.

In [ ]:
numerical_features = X_train.select_dtypes('number').columns.to_list()
print(f'There are {len(numerical_features)} numerical features. They are: {numerical_features}')

Checking categorical features cardinality and unique values.

In [ ]:
categorical_features = X_train.select_dtypes('object').columns.to_list()

print(f'There are {len(categorical_features)} categorical features. They are: {categorical_features}')
print()

for feature in categorical_features:
    print(feature)
    print(f'Number of categories: {X_train[feature].nunique()}. They are:')
    print(X_train[feature].unique())
    print()

I will build the preprocessor below. As I intend to test different algorithms, the same preprocessor will be used for tree-based models and scale sensitive models. Moreover, I will apply the techniques that I've mentioned above.

In [ ]:
target_encoder_features = ['Saving accounts', 'Checking account']
ordinal_encoder_features = ['Purpose', 'Housing']

In [ ]:
ordinal_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder()),
        ('std_scaler', StandardScaler())
        ]
        )

target_pipeline = Pipeline(
    steps=[
        ('target_encoder',TargetEncoder(cols=target_encoder_features)),
        ('std_scaler', StandardScaler())
        ]
        )

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', ordinal_pipeline, ordinal_encoder_features),
        ('target', target_pipeline, target_encoder_features),
        ('std_scaler', StandardScaler(), numerical_features)
        ], remainder='passthrough'
        )

In [ ]:
X_train_prepared = preprocessor.fit_transform(X_train, y_train)
X_train_prepared.shape

# 6. Model training, comparison, selection and tuning
- In this step, I intend to compare different models performances. In order to achieve this, I will use stratified k-fold cross validation to train each one of them and evaluate their ROC-AUC score. Accuracy is not a good metric because the target is imbalanced. Moreover, stratified k-fold cross validation will maintain the target proportion on each fold, dealing with the imbalanced target as well.
- <b>K-fold cross-validation</b> is a technique used in machine learning to assess the performance of a model. It involves dividing the dataset into K subsets, using K-1 for training and one for testing iteratively. This helps in estimating a model's generalization ability by reducing the risk of overfitting and providing more reliable performance metrics.<br>
- My objective here is to select a best model to go for hyperparameter tuning. In order to get this best model, I will evaluate the highest roc-auc average validation scores and look at the bias-variance trade-off.
- After selecting a model, I will tune its hyperparameters.
- <b>Hyperparameter tuning</b> refers to the process of selecting the optimal hyperparameters for a machine learning model. Hyperparameters are parameters that are set before the model is trained and directly influence its performance but are not learned from the data.

Training the models and comparing their performances (roc-auc score) with stratified k-fold cross validation in order to choose one of them for hyperparameter tuning.

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Linear SVM': LinearSVC(),
    'SVM': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier()
}

In [ ]:
eval_df = evaluate_models_cv(models=models, X_train=X_train_prepared, y_train=y_train)

In [ ]:
eval_df.sort_values(['Average Val Score'], ascending=False)

Logistic Regression presents the highest average validation ROC-AUC score. Furthermore, Random Forest is overfitting. There is a significant difference between its validation and training errors. In fact, the training error is zero. Thus, the Random Forest model exhibits low bias and high variance, and since its mean validation score is only slightly lower than that of Logistic Regression, I will choose it for hyperparameter tuning and final model evaluation because there is more room for performance improvement (due to the overfit).

### Hyperparameter tuning
As I intend to tune Random Forest model, I won't use GridSearchCV.

<b>Why not use grid search?</b>
- Using grid search CV to tune Random Forest can be problematic due to its computational expense, limitations in granularity and adaptability, lack of consideration for interaction effects, inefficient exploration of the hyperparameter space, and inefficient allocation of computational resources.
- Grid search exhaustively searches through all combinations of specified hyperparameters, it does not adapt its search based on the observed performance of previous hyperparameter configurations. Also, hyperparameters in Random Forest can have complex interactions, where the impact of one hyperparameter depends on the values of others. Grid search does not explicitly consider these interactions and evaluates hyperparameters independently.
Considering this, I will tune Random Forest using Bayesian Optimization.

<b>What is Bayesian Optimization?</b>
-  Bayesian optimization is an efficient and adaptive technique for finding the optimal combination of hyperparameters for a machine learning model. It uses probabilistic models to intelligently explore the hyperparameter space, balancing exploration and exploitation.

An important point here is to define class_weight hyperparameter. Then, the algorithm will assign different weights for the target's minority (bad risk) and majority (good risk) class instances. Thus, the model will be penalized when mistakenly predicting a bad risk customer and also be able to better learn the patterns in minority class data samples.

In [ ]:
n_folds = 5
stratified_kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

search_space = {
    'n_estimators': Integer(100, 700),
    'criterion': Categorical(['gini', 'entropy', 'log_loss']),
    'max_depth': Integer(2, 16),
    'min_samples_split': Integer(2, 50),
    'min_samples_leaf': Integer(2, 25),
    'bootstrap': Categorical([True, False]),
    'class_weight': Categorical(['balanced', 'balanced_subsample']),
    'max_features': Categorical(['sqrt', 'log2', None])
}

bayesian_search = BayesSearchCV(estimator=RandomForestClassifier(), search_spaces=search_space, cv=stratified_kfold, n_iter=50, scoring='roc_auc', return_train_score=True, random_state=42)

In [ ]:
bayesian_search.fit(X_train_prepared, y_train)

In [ ]:
print(f'The best params fund for Random Forest are: ')
bayesian_search.best_params_

Let's take a look into some results obtained by the bayesian search cv, specially for important parameters like class_weight, criterion, max_depth, min_samples_leaf, min_samples_split and n_estimators.

In [ ]:
bayesian_search_results = pd.DataFrame(bayesian_search.cv_results_)
bayesian_search_results = bayesian_search_results[['rank_test_score', 'mean_test_score',
'mean_train_score', 'param_class_weight',
'param_criterion', 'param_max_depth',
'param_min_samples_leaf', 'param_min_samples_split',
'param_n_estimators']]
bayesian_search_results.sort_values(['rank_test_score']).head(10)

The best model was reached applying regularization! Clearly, bias has increased (higher training error) and thus variance has decreased (lower average validation error), improving the model's generalization ability. It is possible to see that higher values were defined for parameters like min_samples_leaf and min_samples_split and lower values were defined for parameters like max_depth.

# 7. Final production model testing and evaluation
- In this step, I intend to evaluate our final tuned model on test data (simulating production unseen data).
- Once I am facing a binary classification problem with an imbalanced target, recall is the most important metric. I want my model to identify as many as possible bad risk customers. Therefore, we can be more flexible with lower precision values, as false positives (customers who are good risk but were classified as bad risk) will not cause many issues for the bank.
- Moreover, I will look at precision-recall trade-off to see wheter there is space for recall metric improvement without significantly compromising the precision score. This is a good strategy when dealing with imbalanced data.
- By identifying a large portion of the bad risk customers, the bank can make informed decisions, balancing profit generation and prudent risk management and minimizing potential financial losses caused when a borrower fail to pay his credit obligations. This is the main objective of the project.

Getting the final tuned Random Forest model.

In [ ]:
final_rf_clf = bayesian_search.best_estimator_

Using the preprocessor to transform the test predictor set. By applying just transform we avoid data leakage.

In [ ]:
X_test_prepared = preprocessor.transform(X_test)
X_test_prepared.shape

Making predictions.

In [ ]:
final_predictions = final_rf_clf.predict(X_test_prepared)

### Evaluating the final tuned Random Forest model.

In [ ]:
evaluate_classifier(y_test, final_predictions)

<b>The model presents a decent performance!</b>

<b>Recall Score (0.68):</b> The recall score (metric of interest for this analysis), also known as the true positive rate or sensitivity, measures the proportion of actual positive cases (bad risk customers) that the model correctly identifies. The model correctly identifies 68% of the bad risk customers. In practical terms, looking at the confusion matrix, this means that it has defined correctly 41 out of 60 bad risk customers.

<b>Precision Score (0.49):</b> The precision score quantifies the proportion of correctly predicted positive cases (bad risk customers) out of all instances predicted as positive by the model. Out of all the customers predicted as bad risk, 49% of them actually belong to this class. In practical terms, looking at the confusion matrix, this means that from 83 clients predicted as bad risk, 41 of them are actually risky. Thus, the precision score is not so good. However, predicting good risk customers as bad risk (false positives) will not cause many issues for the bank. Our goal is really to correctly identify a significant portion of bad risk customers, and the model does it.

<b>AUC Score (0.69):</b> The AUC (Area Under the Curve) score represents the overall performance of the model in distinguishing between positive (bad risk) and negative (good risk) instances. It measures the probability that the model will rank a randomly chosen positive instance higher than a randomly chosen negative instance. A higher AUC score indicates better performance. With an AUC score of 0.69, the model demonstrates a good ability to differentiate between good and bad risk customers.

### Interpreting the model results with feature importances
I will look at <b>feature importances</b> below, in order to interpret Random Forest results. An important task when performing supervised learning on a dataset is determining which features provide the most predictive power. By focusing on the relationship between only a few crucial features and the target label we simplify our understanding of the phenomenon, which is most always a useful thing to do.

Random Forest calculates feature importances by aggregating the average decrease in impurity (Gini or entropy) across all decision trees in the ensemble. Features that lead to more significant impurity reduction when used for splitting are considered more important. The final importance scores are normalized to sum up to 1 across all features.

In [ ]:
plot_feature_importances(final_rf_clf, df)

Housing, Purpose, Duration, Job and Saving accounts are the 5 features with the highest predictive power. I saw on the eda that customers within some Housing, Purpose, Job and Saving accounts categories like free, vacation/others, highly skilled and little presented a tendency of being any of the credit risk labels (bad or good). Furthermore, I also saw that longer durations credit services are associated with higher credit amounts and thus, higher levels of risk.

Finally, considering this, the model results are consistent and interpretable. Everything makes sense.

### Precision vs Recall trade-off
- My objective in building the model is to correctly predict as many bad risk customers as possible, such that the bank can make informed decisions and minimize risk when offering its credit services. Thus, I will examine the precision-recall curve to determine whether there is room for enhancing the recall metric without significantly compromising the precision score. This is a typical procedure when handling an imbalanced target in binary classification. Finally, I will do the predictions comparing the estimated probabilities of being bad risk (positive) with the threshold that provides the target recall value. If the estimated probability is greater than the threshold, the instance will be classified as positive, else negative.
- <b>Precision-recall trade-off:</b> The precision-recall trade-off involves finding a balance between precision and recall in a classification model. For Random Forest, adjusting the decision threshold can impact this trade-off. Lowering the threshold increases recall but may decrease precision, while raising it has the opposite effect. This trade-off is crucial for optimizing model performance, especially when dealing with imbalanced datasets.

Getting the model's estimated probabilities.

In [ ]:
predicted_probas = final_rf_clf.predict_proba(X_test_prepared)

Plotting precision recall curve for different threshold values.

In [ ]:
precision, recall, threshold = precision_vs_recall_curve(y_test, predicted_probas)

There is space for enhancing the recall metric without significantly compromising the precision score.

Getting the threshold for a 0.8 recall score.

In [ ]:
threshold_precision, threshold_recall, selected_threshold = get_threshold_metrics(precision=precision,
recall=recall, threshold=threshold,
target_metric='recall', target_metric_value=0.8)

Making predictions.

In [ ]:
threshold_predictions = (predicted_probas[:, 1] >= selected_threshold).astype(int)

Evaluating final model after balancing the precision-recall trade-off.

In [ ]:
evaluate_classifier(y_test, threshold_predictions)

The results are great! By dealing with precision-recall trade-off, I was able to select a threshold value that improved the recall metric from 0.68 to 0.8 without compromising the precision score, which fell from 0.49 to 0.48, just a little downgrade! In practical terms, looking at the confusion matrix, it was possible to correctly predict 48 out of 60 bad risk customers. At the same time, I could also improve the ROC-AUC score from 0.69 to 0.71. Finally, the objective was achieved. Now, the bank can correctly identify 80% of the bad risk customers. Thus, my model solves the business problem by allowing the bank to make informed decisions, balancing profit generation with prudent risk management.

### Visualizing the probabilities
Below I get the top 10 customers with respect to highest and lowest Random Forest model's estimated probabilities of being bad risk. As we can see, the model results are coherent! Sometimes it makes errors, but mostly it is correct with respect to risky clients.

Top 10 customers with highest model's estimated probabilities of being bad risk.

In [ ]:
bad_risk_probas = predicted_probas[:, 1]

probas_df = pd.concat([X_test, y_test], axis=1)
probas_df['Bad Risk Probas'] = bad_risk_probas
probas_df.sort_values(['Bad Risk Probas'], ascending=False).head(10)

Top 10 customers with lowest model's estimated probabilities of being bad risk.

In [ ]:
probas_df.sort_values(['Bad Risk Probas'], ascending=True).head(10)

The model's estimated probabilities make sense!
- Looking at the two customers with highest probabilities of being bad risk, they don't have a house, present little Saving account and borrowed a high credit amount for a long duration. Moreover, both of them did it for business purposes, configuring higher levels of risk! Business is about risk taking and, since they have a low balance saving account, they were correctly classified as bad risk with some certainty.
- Looking at the customers with lowest model's estimated probabilities of being bad risk, it is clear that, having a house, taking lower credit amounts for shorter durations for car or radio/tv purposes can configure a client as good risk.
- The insights found in the EDA step and feature importances were confirmed by the model's estimated probabilities.


# 8. Conclusions
- In this project, I have built a Random Forest Classifier model for predicting credit risk of German Bank's customers. The objective of the project was to build a model that was able to correctly identify as many as possible bad risk customers, focusing on the recall score. By doing this, the bank can make informed decisions, balancing profit generation with prudent risk management and minimizing potential financial losses caused when a borrower fails to repay a loan or a credit obligation.
- The business problem is solved now. My model is able to correctly predict 80% of the bad risk customers! Furthermore, the eda step provided a lot of useful insights that helped us understand the problem and credit risk patterns within the available features.
- The main techniques used to deal with the imbalanced target were stratified split, stratified k-fold cross validation, class weight Random Forest model hyperparameter and precision-recall trade-off balancing. I didn't intend to use SMOTE, because it is prone to overfitting and a lot of experient data scientists don't recommend it.
- First, I have reached a 0.68 recall score by applying hyperparameter tuning on Random Forest. Then, I looked at the precision-recall trade-off, finding a threshold that provided a recall of 0.8. I made the predictions again, now comparing the model's estimated probabilities of the customer being bad risk with this threshold. If the probability was greater than it, the instance was classified as positive, else negative. By doing this, the sensitivity in fact went from 0.68 to 0.8 without compromising the precision metric, which fell from 0.49 to 0.48.
- The model results were interpreted and they are coherent! By looking at feature importances, it was possible to assess the features with the highest predictive power for the analysis. In the EDA step, I had already thought these features would be the most useful, reinforcing that the estimator outcomes make sense. Finally, the model's estimated probabilities also make sense and thus everything is working fine.
- The next step is to deploy the model using CI/CD pipeline principles, with modular coding.